**C2_W2_Horse_vs_Humans_Augmentation**

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip



--2020-09-09 15:43:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.198.128, 209.85.200.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.198.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   107MB/s    in 1.3s    

2020-09-09 15:43:22 (107 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2020-09-09 15:43:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.129.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.129.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length

In [3]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [4]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')
train_human_dir = os.path.join('/tmp/horse-or-human/humans')
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')


In [5]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [7]:
model = tf.keras.models.Sequential([
                                   tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Flatten(),
                                   tf.keras.layers.Dense(512, activation='relu'),
                                   tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory('/tmp/horse-or-human/',
                                                    target_size=(300, 300),
                                                    batch_size=128, 
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory('/tmp/validation-horse-or-human/',
                                                        target_size=(300, 300),
                                                        batch_size=128,
                                                        class_mode='binary')


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    validation_steps = 8,
                    steps_per_epoch = 8,
                    epochs= 100,
                    verbose=1)

Epoch 1/100
8/8 [==============================] - 20s 3s/step - loss: 0.6862 - accuracy: 0.5384 - val_loss: 0.6671 - val_accuracy: 0.6680
Epoch 2/100
8/8 [==============================] - 20s 3s/step - loss: 0.6725 - accuracy: 0.6096
Epoch 3/100
8/8 [==============================] - 23s 3s/step - loss: 0.6681 - accuracy: 0.6730
Epoch 4/100
8/8 [==============================] - 21s 3s/step - loss: 0.6341 - accuracy: 0.7041
Epoch 5/100
8/8 [==============================] - 20s 3s/step - loss: 0.6276 - accuracy: 0.6941
Epoch 6/100
8/8 [==============================] - 24s 3s/step - loss: 0.5594 - accuracy: 0.7529
Epoch 7/100
8/8 [==============================] - 20s 3s/step - loss: 0.5982 - accuracy: 0.6774
Epoch 8/100
8/8 [==============================] - 24s 3s/step - loss: 0.5252 - accuracy: 0.7617
Epoch 9/100
8/8 [==============================] - 23s 3s/step - loss: 0.5509 - accuracy: 0.7208
Epoch 10/100
8/8 [==============================] - 20s 3s/step - loss: 0.5148 - accu

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs=range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(eochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(eochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()